In [1]:
import sys
import pathlib
import pandas as pd
import json
import os
import numpy as np
from PIL import Image
import tensorflow as tf


In [2]:
sys.path.append(os.path.join(os.getcwd(), 'efficientdetv2', 'automl'))
sys.path.append(os.path.join(os.getcwd(), 'efficientdetv2', 'automl', 'efficientnetv2'))
sys.path.insert(0, os.path.join(os.getcwd(), 'efficientdetv2', 'automl', 'efficientdet'))

In [3]:
import hparams_config
import inference
from keras import efficientdet_keras

In [4]:
model_name = 'efficientdetv2-s'
images_dir = r'D:\Datasets\siim_covid19_detection\1080px\train\image'
val_json_dir = r'D:\Datasets\siim_covid19_detection\1080px\object_detection_files\files_fold_2\valid'
# output_dir = 
model_dir = r'C:\Users\prans\Python files\Kaggle Competitions\Covid_19_object_detection\checkpoints\siim\detection\efficientdetv2-s\512px\model_1'

In [5]:
val_image_json = os.path.join(val_json_dir, 'images_info.json')
val_object_json =  os.path.join(val_json_dir, 'object_detection_info.json')

In [6]:
with open(val_image_json, 'r') as image_info_file:
    image_info = json.load(image_info_file)
with open(val_object_json, 'r') as det_info_file:
    det_info = json.load(det_info_file)

In [7]:
image_info['images'][0]

{'id': 1, 'file_name': '000c3a3f293f_image.png', 'height': 1080, 'width': 1080}

In [8]:
det_info['images'][0]

{'id': 1, 'file_name': '000c3a3f293f_image.png', 'height': 1080, 'width': 1080}

In [9]:
det_info['annotations'][0]

{'image_id': 2,
 'bbox': [239.40311937172774,
  84.04667830188679,
  306.68242264397907,
  424.43581415094343],
 'area': 0.11,
 'category_id': 1,
 'id': 1}

In [10]:
config = hparams_config.get_efficientdet_config(model_name)
config.is_training_bn = False
config.image_size = 512
config.nms_configs.score_thresh = 0.5
config.nms_configs.max_output_size = 10


In [11]:
model = efficientdet_keras.EfficientDetModel(config = config)
model.build((None,512, 512, 3))

C:\Users\prans\Python files\Kaggle Competitions\Covid_19_object_detection\env\lib\site-packages\tensorflow\python\keras\engine\base_layer.py:1331: UserWarning: `layer.updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`layer.updates` will be removed in a future version. '


In [12]:
model.load_weights(model_dir)
model.summary()

Model: ""
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnetv2-s (EffNetV2Mo [(None, 1280), (None, Non 20331360  
_________________________________________________________________
resample_p6 (ResampleFeature multiple                  16704     
_________________________________________________________________
resample_p7 (ResampleFeature multiple                  0         
_________________________________________________________________
fpn_cells (FPNCells)         multiple                  174393    
_________________________________________________________________
class_net (ClassNet)         multiple                  19794     
_________________________________________________________________
box_net (BoxNet)             multiple                  20964     
Total params: 20,563,215
Trainable params: 20,401,791
Non-trainable params: 161,424
________________________________________________

In [13]:
# Read the image
img = tf.io.read_file(os.path.join(images_dir, det_info['images'][0]['file_name']))
# Decode to png
img = tf.io.decode_png(img)
# Convert to RGB
img = tf.image.grayscale_to_rgb(img)
# Expand the dims to include batch_size dim
img = tf.expand_dims(img,0)
# Convert to float32 dtype
img = tf.cast(img, dtype = tf.float32)

In [15]:
# Make the prediction and return boxes, scores, classes
classes, boxes = model(img, training = False, pre_mode = 'infer',  post_mode = 'global')

In [ ]:
valid_len

In [16]:
boxes

[<tf.Tensor: shape=(1, 135, 135, 36), dtype=float32, numpy=
 array([[[[  8.047605  ,  -3.3287807 ,   5.9023857 , ...,   6.434737  ,
            -3.2039888 ,  14.19239   ],
          [  9.459911  ,   3.3434463 ,   6.109568  , ...,   3.531522  ,
            -7.0199733 ,  18.80606   ],
          [ 16.386965  ,   6.376075  ,  19.789581  , ...,   6.8959184 ,
            -2.2804575 ,  28.423712  ],
          ...,
          [ 23.325844  ,  -4.052578  , -10.543217  , ...,   8.815443  ,
            -2.5514302 ,  14.747911  ],
          [ 14.218494  ,   9.155863  , -12.186209  , ...,   9.068143  ,
           -10.11868   ,   4.2021637 ],
          [  5.393466  ,   5.2157173 ,   1.7458203 , ...,  -0.96191823,
            -3.3463614 ,  -2.3528426 ]],
 
         [[ 10.338584  ,  -2.2233078 ,   4.669423  , ...,  15.281514  ,
            -4.461145  ,   2.16498   ],
          [ 10.550395  ,  10.927393  ,   5.939534  , ...,  14.345527  ,
           -12.650906  ,   7.133545  ],
          [ 14.425691  ,  

In [24]:
tf.nn.sigmoid(tf.reshape(classes[0][0,0,0,:], (-1,2)))

<tf.Tensor: shape=(9, 2), dtype=float32, numpy=
array([[1.3400958e-35, 1.8222273e-30],
       [4.2144368e-35, 5.5606953e-26],
       [5.5592544e-33, 1.8539081e-27],
       [1.2244784e-37, 7.0379937e-25],
       [1.4257024e-36, 8.1787251e-29],
       [3.6107480e-33, 3.5053442e-28],
       [6.3047657e-28, 9.0565772e-30],
       [1.0989238e-27, 1.6607060e-25],
       [1.7428662e-29, 5.1957744e-31]], dtype=float32)>